---
#1. Preparation
##1.1 Download Stable-Diffusion Checkpoint

In [ ]:
%%shell
mkdir -p pretrained/
cd pretrained/
wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt
cd ..

##1.2 Clone CelebBasis Code from [GitHub](https://github.com/ygtxr1997/CelebBasis/tree/main)

In [ ]:
!git clone https://github.com/ygtxr1997/CelebBasis.git

## 1.3 Download the StyleGAN Datasets and Third-Party Pretrained Weights from HuggingFace

In [ ]:
# dataset
!mkdir -p ./datasets/ && cd ./datasets/ && wget -q https://huggingface.co/datasets/ygtxr1997/CelebBasis/resolve/main/stylegan.tar
!cd ./datasets/ && tar -xf stylegan.tar && rm stylegan.tar
# face alignment model
!mkdir -p ./pretrained/PIPNet/ && mkdir -p ./pretrained/glint360k_cosface_r100_fp16_0.1/
!cd ./pretrained/PIPNet/ && wget -q https://huggingface.co/datasets/ygtxr1997/CelebBasis/resolve/main/PIPNet/FaceBoxesV2.pth && wget -q https://huggingface.co/datasets/ygtxr1997/CelebBasis/resolve/main/PIPNet/epoch59.pth
!cp ./pretrained/PIPNet/* ./CelebBasis/evaluation/face_align/PIPNet/weights
# face recognition model
!cd ./pretrained/glint360k_cosface_r100_fp16_0.1/ && wget -q https://huggingface.co/datasets/ygtxr1997/CelebBasis/resolve/main/glint360k_cosface_r100_fp16_0.1/backbone.pth
!mkdir -p ./CelebBasis/weights/
!cp -r ./pretrained/glint360k_cosface_r100_fp16_0.1/ ./CelebBasis/weights/

##1.4 (Optional) Copy StyleGAN Datasets and Third-Party Pretrained Weights from Your Own Google Drive
*If you have any downloading problem in (Step 1.3), you can copy the datasets and pretrained models to your own Google Drive as follows.*

You should first copy [CelebBasis Google Drive](https://drive.google.com/drive/folders/181pJ3Y2Ffk0R8jVcErPT611aWQLCz7D9?usp=sharing) to your own [Google Drive](https://drive.google.com/drive/my-drive).
The Google Drive directory tree will be like:
```shell
My Drive/
 |--CelebBasis/
  |--datasets/
  |--pretrained/
```
Then, the following cell mounts your own Google Drive to this Colab Notebook and copies the datasets and weights files from the mounted folder.

In [ ]:
# @title Use Googgle Drive? { display-mode: "form" }

use_google_drive = False # @param ["False", "True"]
print('Use Google Drive:', use_google_drive)
if use_google_drive:
  import os
  from google.colab import drive
  drive.mount('/content/drive')
  os.system('cp -r /content/drive/MyDrive/CelebBasis/datasets/ ./')
  os.system('cp -r /content/drive/MyDrive/CelebBasis/pretrained/* ./pretrained/')
  drive.flush_and_unmount()
  os.system('cd ./datasets && tar -xf stylegan.tar && rm stylegan.tar')
  os.system('cp ./pretrained/PIPNet/* ./CelebBasis/evaluation/face_align/PIPNet/weights')
  os.system('mkdir -p ./CelebBasis/weights/')
  os.system('cp -r ./pretrained/glint360k_cosface_r100_fp16_0.1/ ./CelebBasis/weights/')
else:
  pass

## 1.5 Use PIP to Install the Python Dependencies and Uninstall the Needless Modules (which are preinstalled by Google Colab but not used)

In [ ]:
!mkdir -p pip_installed/ && cd pip_installed/ && pip install -r ../CelebBasis/requirements.txt
!pip uninstall -r ./CelebBasis/requirements_uninstall.txt -y
!pip list |grep torch

---
#2. Crop and Align Faces

In [ ]:
!cd CelebBasis && chmod +x *.sh && bash ./00_align_face.sh ../datasets/stylegan/stylegan3-r-ffhq-1024x1024 ../datasets/stylegan/ffhq

#3. Train
* Limited by Google Colab GPU Memory ($<$16GB), we have to modify the `configs/stable-diffusion/aigc_id_colab.yaml` as follows:
```yaml
model:
  params:
    personalization_config:
      params:
        momentum: 0.9  # default: 0.99
data:
  params:
    batch_size: 1  # default: 2
    num_workers: 2  # default: 8
lightning:
  modelcheckpoint:
    params:
      every_n_train_steps: 400  # default: 200
  callbacks:
    image_logger:
      params:
        batch_frequency: 1200  # default: 600
  trainer:
    max_steps: 800  # default: 1600
```
* These modifications may increase the training time and downgrade the performance slightly.

In [ ]:
!cd CelebBasis && bash ./01_start_train.sh ../pretrained/sd-v1-4-full-ema.ckpt ./configs/stable-diffusion/aigc_id_colab.yaml

#4. Test
Modify the following `"traininYYYY-MM-DDTHH-MM-SS_celebbasis/"` to the project folder under `./CelebBasis/logs/` and run the test.

The generated images will be output under `./CelebBasis/outputs/traininYYYY-MM-DDTHH-MM-SS_celebbasis/`. Enjoy it!

In [ ]:
!cd CelebBasis && bash ./02_start_test.sh "../pretrained/sd-v1-4-full-ema.ckpt" "./infer_images/example_prompt.txt" "training2023-07-13T14-55-56_celebbasis" 4

In [ ]:
!nvidia-smi